In [1]:
import folium
import pandas as pd
import base64
from io import BytesIO

## Distrito Federal location

In [2]:
mapa = folium.Map(location=[-15.7836, -47.7750])

## encoding image to use local

In [3]:
encoded = base64.b64encode(open('assets/Group.png', 'rb').read())
decoded = base64.b64decode(encoded)

## Adding hospitals from CSV file

In [4]:
df_hospitais = pd.read_csv('data/hospitais-brasilia.csv')
#couldn't encode this one
hospital_image_url = 'https://github.com/chris-redfield/hacksaude-2018/raw/master/assets/hostpital-building.png'
for index, row in df_hospitais.iterrows():
    
    icon_url = BytesIO(decoded)
    
    como_chegar = 'https://www.google.com/maps/dir/?api=1&origin=current+location&destination='\
    + str(row['lat']) + ',' + str(row['long'])
    
    html = '<h5 style="margin-bottom:6px">' + row['nome'] + '</h5><hr style="margin-top:0px">' + '<img src="'+hospital_image_url \
    +'" style="float:left;padding-right:19px">'\
    'Profissionais: '\
    + str(row['qtd_profissionais']) + '<br>' + 'Horário: ' +row['atendimento']\
    + '<br>' + 'Fone: ' + row['telefone'] + '<br> <a target="_blank" href="'+como_chegar+'">Como Chegar</a>'\
    + '<br><br> Endereço: ' + row['endereco']
    
    iframe = folium.IFrame(html=html, width=280, height=195)
    popup = folium.Popup(iframe, max_width=2650)
    
    marker = folium.Marker([row['lat'],row['long']],
                  popup=popup,
                  icon=folium.CustomIcon(icon_url,icon_size=(32,41),icon_anchor=(16,30)),
                 ).add_to(mapa)
    
    folium.CircleMarker([row['lat'],row['long']],
                    radius=row['qtd_profissionais']/150,
                    color='#3186cc',
                    fill=True,
                    fill_color='#3186cc',
                    fillOpacity= 0.8,
                   ).add_to(mapa)
mapa

## Adding UBS'

In [5]:
from folium.plugins import MarkerCluster

marker_cluster = MarkerCluster().add_to(mapa)

df_ubs = pd.read_csv('data/UBS-no-DF-abril-2018-v2.csv')
qtd_profissionais = 800
for index, row in df_ubs.iterrows():

    icon_url = BytesIO(decoded)
    
    como_chegar = 'https://www.google.com/maps/dir/?api=1&origin=current+location&destination='\
    + str(row['lat']) + ',' + str(row['long'])
    
    html = '<h5 style="margin-bottom:6px">' + row['UNIDADE BÁSICA DE SAÚDE'] + '</h5>' \
    + '<hr style="margin-top:0px">' + '<img src="'+ hospital_image_url+'"' \
    + 'style="float:left;padding-right:19px">' \
    + 'Horário: ' +row['HORÁRIO DE FUNCIONAMENTO']\
    + '<br>' + 'Fone: ' + row['telefone']\
    + '<br> <a target="_blank" href="'+como_chegar+'">Como Chegar</a>'\
    + '<br><br><br> Endereço: ' + row['ENDEREÇO DA UNIDADE']
    iframe = folium.IFrame(html=html, width=280, height=210)
    popup = folium.Popup(iframe, max_width=2650)    
    
    marker = folium.Marker([row['lat'],row['long']],
                  popup=popup,
                  icon=folium.CustomIcon(icon_url,icon_size=(24,31),icon_anchor=(12,23))
                 ).add_to(marker_cluster)
mapa

In [6]:
mapa.save('maps/mapa-hospitais-e-ubs.html')